In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np

df = pd.read_csv("./data/train.csv", sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
ids = np.arange(0, df.shape[0])
df["ClientID"] = ids
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,ClientID
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,2
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,3
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,4


In [4]:
client_id_colname = 'ClientID'
SHUFFLE_BUFFER = 1000
NUM_EPOCHS = 1
# split client id into train and test clients
client_ids = df[client_id_colname].unique()
train_client_ids = pd.DataFrame(client_ids).sample(frac=0.5).values.tolist()
test_client_ids = [x for x in client_ids if x not in train_client_ids]

def create_tf_dataset_for_client_fn(client_id):
  # a function which takes a client_id and returns a
  # tf.data.Dataset for that client
  client_data = df[df[client_id_colname] == client_id[0]]
  dataset = tf.data.Dataset.from_tensor_slices(client_data.fillna('').to_dict("list"))
  dataset = dataset.shuffle(SHUFFLE_BUFFER).batch(1).repeat(NUM_EPOCHS)
  return dataset

train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=train_client_ids,
        serializable_dataset_fn=create_tf_dataset_for_client_fn
    )
example_dataset = train_data.create_tf_dataset_for_client(
        train_data.client_ids[0]
    )

print(type(example_dataset))
example_element = iter(example_dataset).next()
print(example_element)

<class 'tensorflow.python.data.ops.dataset_ops.RepeatDataset'>
{'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([59], dtype=int32)>, 'job': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'retired'], dtype=object)>, 'marital': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'married'], dtype=object)>, 'education': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'tertiary'], dtype=object)>, 'default': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'no'], dtype=object)>, 'balance': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([135], dtype=int32)>, 'housing': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'yes'], dtype=object)>, 'loan': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'no'], dtype=object)>, 'contact': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'unknown'], dtype=object)>, 'day': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([17], dtype=int32)>, 'month': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'nov'], dtype=object

2023-03-15 11:19:31.766689: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-15 11:19:31.766954: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-15 11:19:31.766973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dhiraj-Inspiron-5593): /proc/driver/nvidia/version does not exist
2023-03-15 11:19:31.768399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [32]:
example_element.values()

dict_values([<tf.Tensor: shape=(1,), dtype=int32, numpy=array([59], dtype=int32)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'retired'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'married'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'tertiary'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'no'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([135], dtype=int32)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'yes'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'no'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'unknown'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([17], dtype=int32)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'nov'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([94], dtype=int32)>, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, <tf.Tensor: sh

In [26]:
client_data = df[df[client_id_colname] == 1]
client_data.fillna('').to_dict("list")

{'age': [44],
 'job': ['technician'],
 'marital': ['single'],
 'education': ['secondary'],
 'default': ['no'],
 'balance': [29],
 'housing': ['yes'],
 'loan': ['no'],
 'contact': ['unknown'],
 'day': [5],
 'month': ['may'],
 'duration': [151],
 'campaign': [1],
 'pdays': [-1],
 'previous': [0],
 'poutcome': ['unknown'],
 'y': ['no'],
 'ClientID': [1]}